In [4]:
from transformers import AutoTokenizer, AutoModel
from torch import nn

tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")
model = AutoModel.from_pretrained("bert-base-uncased")

inputs = tokenizer("Hello world!", return_tensors="pt")
outputs = model(**inputs)
outputs

Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "/home/ubutnu/anaconda3/envs/py38/lib/python3.8/site-packages/urllib3/connection.py", line 174, in _new_conn
  File "/home/ubutnu/anaconda3/envs/py38/lib/python3.8/site-packages/urllib3/util/connection.py", line 95, in create_connection
  File "/home/ubutnu/anaconda3/envs/py38/lib/python3.8/site-packages/urllib3/util/connection.py", line 85, in create_connection
OSError: [Errno 101] Network is unreachable

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/ubutnu/anaconda3/envs/py38/lib/python3.8/site-packages/urllib3/connectionpool.py", line 714, in urlopen
    CertificateError,
  File "/home/ubutnu/anaconda3/envs/py38/lib/python3.8/site-packages/urllib3/connectionpool.py", line 403, in _make_request
    # timeouts, check for a zero timeout before making the request.
  File "/home/ubutnu/anaconda3/envs/py38/lib/python3.8/site-packages/urllib3/connectionpool.py", line 1053, in _

In [1]:
from datasets import load_dataset
dataset = load_dataset('oscar', 'unshuffled_deduplicated_it',cache_dir='~/hardDisk/DeepLearning/bert',)
dataset['train']

Using the latest cached version of the module from /home/ubutnu/.cache/huggingface/modules/datasets_modules/datasets/oscar/84838bd49d2295f62008383b05620571535451d84545037bb94d6f3501651df2 (last modified on Thu Sep 14 16:34:14 2023) since it couldn't be found locally at oscar., or remotely on the Hugging Face Hub.
Found cached dataset oscar (/home/ubutnu/hardDisk/DeepLearning/bert/oscar/unshuffled_deduplicated_it/1.0.0/84838bd49d2295f62008383b05620571535451d84545037bb94d6f3501651df2)


  0%|          | 0/1 [00:00<?, ?it/s]

Dataset({
    features: ['id', 'text'],
    num_rows: 28522082
})

In [7]:
from tqdm.auto import tqdm 
 
text_data = [] 
file_count = 0 
 
for sample in tqdm(dataset['train']): 
    sample = sample['text'].replace('\n', '') 
    text_data.append(sample) 
    if len(text_data) == 10_000: 
        # once we git the 10K mark, save to file 
        with open(f'/home/ubutnu/hardDisk/DeepLearning/data/text/oscar_it/text_{file_count}.txt', 'w', encoding='utf-8') as fp: 
            fp.write('\n'.join(text_data)) 
        text_data = [] 
        file_count += 1 
# after saving in 10K chunks, we will have ~2082 leftover samples, we save those now too 


  0%|          | 0/28522082 [00:00<?, ?it/s]

FileNotFoundError: [Errno 2] No such file or directory: '../../data/text/oscar_it/text_2852.txt'

In [6]:
from pathlib import Path 
paths = [str(x) for x in Path('../../data/text/oscar_it').glob('**/*.txt')] 
from tokenizers import ByteLevelBPETokenizer 
tokenizer = ByteLevelBPETokenizer() 
tokenizer.train(files=paths[:5], vocab_size=30_522, min_frequency=2, 
                special_tokens=['<s>', '<pad>', '</s>', '<unk>', '<mask>'])

MultiHeadAttention(
  (attention): DotProductAttention(
    (dropout): Dropout(p=0.5, inplace=False)
  )
  (W_q): Linear(in_features=100, out_features=100, bias=False)
  (W_k): Linear(in_features=100, out_features=100, bias=False)
  (W_v): Linear(in_features=100, out_features=100, bias=False)
  (W_o): Linear(in_features=100, out_features=100, bias=False)
)

In [7]:
batch_size, num_queries = 2, 4
num_kvpairs, valid_lens =  6, torch.tensor([3, 2])
X = torch.ones((batch_size, num_queries, num_hiddens))
Y = torch.ones((batch_size, num_kvpairs, num_hiddens))
attention(X, Y, Y, valid_lens).shape

torch.Size([2]) tensor([3, 2])


torch.Size([2, 4, 100])

In [8]:
def predict_ch8(prefix, num_preds, net, vocab, device):  #@save
    """在prefix后面生成新字符"""
    state = net.begin_state(batch_size=1, device=device)
    outputs = [vocab[prefix[0]]]
    get_input = lambda: torch.tensor([outputs[-1]], device=device).reshape((1, 1))
    for y in prefix[1:]:  # 预热期
        _, state = net(get_input(), state)
        outputs.append(vocab[y])
    for _ in range(num_preds):  # 预测num_preds步
        y, state = net(get_input(), state)
        outputs.append(int(y.argmax(dim=1).reshape(1)))
    return ''.join([vocab.idx_to_token[i] for i in outputs])

In [9]:
def grad_clipping(net, theta):  #@save
    """裁剪梯度"""
    if isinstance(net, nn.Module):
        params = [p for p in net.parameters() if p.requires_grad]
    else:
        params = net.params
    norm = torch.sqrt(sum(torch.sum((p.grad ** 2)) for p in params))
    if norm > theta:
        for param in params:
            param.grad[:] *= theta / norm
def train_epoch_ch8(net, train_iter, loss, updater, device, use_random_iter):
    """Train a net within one epoch (defined in Chapter 8).

    Defined in :numref:`sec_rnn_scratch`"""
    state, timer = None, d2l.Timer()
    metric = d2l.Accumulator(2)  # Sum of training loss, no. of tokens
    for X, Y in train_iter:
        if state is None or use_random_iter:
            # Initialize `state` when either it is the first iteration or
            # using random sampling
            state = net.begin_state(batch_size=X.shape[0], device=device)
        else:
            if isinstance(net, nn.Module) and not isinstance(state, tuple):
                # `state` is a tensor for `nn.GRU`
                state.detach_()
            else:
                # `state` is a tuple of tensors for `nn.LSTM` and
                # for our custom scratch implementation
                for s in state:
                    s.detach_()
        y = Y.T.reshape(-1)
        X, y = X.to(device), y.to(device)
        y_hat, state = net(X, state)
        print(y_hat.size())
        l = loss(y_hat, y.long()).mean()
        if isinstance(updater, torch.optim.Optimizer):
            updater.zero_grad()
            l.backward()
            grad_clipping(net, 1)
            updater.step()
        else:
            l.backward()
            grad_clipping(net, 1)
            # Since the `mean` function has been invoked
            updater(batch_size=1)
        metric.add(l * d2l.size(y), d2l.size(y))
    return math.exp(metric[0] / metric[1]), metric[1] / timer.stop()
#@save
def train_ch8(net, train_iter, vocab, lr, num_epochs, device,
              use_random_iter=False):
    """训练模型（定义见第8章）"""
    loss = nn.CrossEntropyLoss()
    animator = d2l.Animator(xlabel='epoch', ylabel='perplexity',
                            legend=['train'], xlim=[10, num_epochs])
    # 初始化
    if isinstance(net, nn.Module):
        updater = torch.optim.SGD(net.parameters(), lr)
    else:
        updater = lambda batch_size: d2l.sgd(net.params, lr, batch_size)
    predict = lambda prefix: predict_ch8(prefix, 50, net, vocab, device)
    # 训练和预测
    for epoch in range(num_epochs):
        ppl, speed = train_epoch_ch8(
            net, train_iter, loss, updater, device, use_random_iter)
        if (epoch + 1) % 10 == 0:
            print(predict('time traveller'))
            animator.add(epoch + 1, [ppl])
    print(f'困惑度 {ppl:.1f}, {speed:.1f} 词元/秒 {str(device)}')
    print(predict('time traveller'))
    print(predict('traveller'))

In [30]:
x=[[[1,2,3],[2,3,4]],[[1,2,3],[2,3,4]],[[1,2,3],[2,3,4]],[[1,2,3],[2,3,4]]]
dec_attention_weights_2d = [head for blk in x  for head in blk]
torch.tensor(x).shape,dec_attention_weights_2d


(torch.Size([1, 2, 3]), [[1, 2, 3], [2, 3, 4]])

In [67]:
import random
import os
def _read_wiki(data_dir):
    file_name = os.path.join(data_dir, 'wiki.train.tokens')
    with open(file_name, 'r') as f:
        lines = f.readlines()
    # 大写字母转换为小写字母
    paragraphs = [line.strip().lower().split(' . ')
                  for line in lines if len(line.split(' . ')) >= 2]
    random.shuffle(paragraphs)
    return paragraphs

data_dir = d2l.download_extract('wikitext-2', 'wikitext-2')
paragraphs = _read_wiki(data_dir)
paragraphs = [d2l.tokenize(
    paragraph, token='word') for paragraph in paragraphs]
# sentences = [sentence for paragraph in paragraphs
#                 for sentence in paragraph]
for i in range(2):
    print(paragraphs[i])


["in 1940 , the barker 's live @-@ in maid retired , and dorothy barker closed her school at the back of the house in the waldrons", 'she continued to supervise the household , and to give both her mother and sister the care they needed', "dorothy and her sister collaborated upon only two books : our darling 's first book and the christian @-@ themed , he leadeth me", 'in 1954 dorothy barker died of a heart attack', "barker was unable to pursue her art to any significant extent following her sister 's death , as all the care of her aged mother devolved upon her , but she did manage to begin planning a stained glass window design in her sister 's memory for st. edmund 's , pitlake ."]
['in latin america , an exclusive version of god of war saga , titled god of war : <unk> collection , was released in november 2012', 'the <unk> collection features three blu @-@ ray discs , as opposed to two , with chains of olympus and ghost of sparta included on the third disc', 'it also includes a <unk